# 3. Getting Started with CrewAI
Welcome to the 3th exercise in the Cloudera Agentic AI Hands on Lab. In this notebook you will get familiar the CrewAI library and how we can leverage it with Cloudera's AI Inference Service. This notebook is designed to walk you through the setup and application of these tools in an agentic worfklow context to create a robust and intelligence system. 

- **Configure AI Inference:** A private Meta LLama 3.1 8B model will serve as the backbone of our AI agents, providing the computational power and linguistic understanding necessary for processing complex queries.
- **Set up CrewAI Agents and Tasks:** We will leverage the CrewAI library to set up our AI agents with clear roles, backstories and goals. We will also configure tasks for each agent to complete and identify tools that they can leverage to complete their tasks.
- **Create Sequential Agentic Workflow:** Next, we will orchestrate these agents and tasks into a crew and workflow for them to complete.
- **Running a Practical Example:** We will then run our example workflow by providing inputs for the agentic workflow to consider and then complete the tasks they are configured to do.

## 3.1 Configuring AI Inference

Let's start by configuring AI Inference's Llama 3.1 8B model as the Base Model we will use for the agents.

In [1]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool
import os
import litellm
import json
import warnings
warnings.filterwarnings('ignore')

2025-03-13 01:46:35,991 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.31.1.


/home/cdsw/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/cdsw/.local/lib/python3.10/site-packages/crewai_tools/tools/base_tool.py:28: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("args_schema", always=True, pre=True)
/home/cdsw/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDic

Below, we obtain the JWT token to use with the Inference Service and configure LiteLLM to work with our OpenAI-compatible endpoint.

In [ ]:
OPENAI_API_KEY=json.load(open("/tmp/jwt"))["access_token"]

In [ ]:
litellm.set_verbose=False

llm = LLM(
    model="openai/" + os.environ["OPENAI_MODEL_ID"],
    api_key=OPENAI_API_KEY,
    base_url=os.environ["OPENAI_BASE_URL"]
)

## 3.2 Set Up CrewAI Agents and Tasks

Next, let's set up the relevant Agents and Tasks using the CrewAI library. We will define 2 agents - One Researcher and One Reporting Analyst, and they will be tasked to research the topic assigned to them. For the purpose of this exercise, we will use the SerperDevTool to access the internet.

In [4]:
from textwrap import dedent

search_tool = SerperDevTool(
    country="us",
    locale="en",
    location="Miami Beach, Florida, United States",
    n_results=5,
)

# First we define the Researcher Agent
researcher_agent = Agent(
    role=dedent((
        """
        {topic} Senior Data Researcher
        """)), # Think of this as the job title
    backstory=dedent((
        """
        You are a seasoned researcher with a knack for uncovering the latest developments in {topic}.

        You are known for your ability to find the most relevant information and present it in a clear and concise manner.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        Uncover cutting-edge developments in {topic} and use the search_tool available to you to look up the latest information.
        
        Try to keep final answers in markdown format.
        """)), # This is the goal that the agent is trying to achieve
    tools=[search_tool],
    allow_delegation=False,
    max_iter=2,
    max_retry_limit=3,
    max_retries=3,
    llm=llm,
    verbose=True
)

# Next, we define the Reporting Analyst
reporting_analyst = Agent(
    role=dedent((
        """
        {topic} Reporting Analyst
        """)),
    backstory=dedent((
        """
        You are a meticulous analyst with a keen eye for detail. You are known for your ability to turn complex data into clear and concise reports,
        making it easy for others to understand and act on the information you provide.
        """)),
    goal=dedent((
        """
        Create detailed reports based on {topic} data analysis and research findings.
        """)), # This is the goal that the agent is trying to achieve
    allow_delegation=False,
    max_iter=2,
    max_retry_limit=3,
    max_retries=3,
    llm=llm,
    verbose=True
)

Next, let's define the tasks for our agents to complete.

In [5]:
# First, let's create a task for the researcher agent. Note below that we define which agent completes which task.
research_task = Task(
    description=dedent((
        """
        Conduct thorough research about {topic}

        Make sure you find any interesting and relevant information given the current year is 2025.
        """)),
    expected_output=dedent((
        """
        A list with 2 to 4 bullet points of the most relevant information about {topic}
        """)),
    agent=researcher_agent
)

reporting_task = Task(
    description=dedent((
        """
        Review the context you got and expand each topic into a full section for a report.
        Make sure the report is detailed and contains any and all relevant information.
        """)),
    expected_output=dedent((
        """
        A fully fledge reports with the mains topics, each with a full section of information.
        Formatted as markdown without '```'
        """)),
    agent=reporting_analyst,
    output_file='report.md',
    context=[research_task]
)

## 3.3 Create Sequential Agentic AI Workflow

Now, let's put all of this together by placing the discrete agents and tasks into a crew.

In [6]:
crew = Crew(
        agents=[researcher_agent, reporting_analyst],
        tasks=[research_task, reporting_task],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
    )

## 3.4 Running a Practical Example

Now, let's put all of this together by placing the discrete agents and tasks into a crew. First, we define an input topic. Feel free to change the topic to something else.

In [7]:
# Notice here that the object we pass maps to the {topic} field we added to the agents and tasks.
inputs = {
        "topic": "Top 5 Cities in Europe",
}

Next, we create a function to run the crew with the given input. We have enabled verbose logging to help you see the agents execute each task.

In [8]:
def run_crew():
    result = crew.kickoff(inputs=inputs)
run_crew()

# Agent: 
## Task: 
Conduct thorough research about Top 5 Cities in Europe

Make sure you find any interesting and relevant information given the current year is 2025.



# Agent: 
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Top 5 Cities in Europe 2025\"}"
## Tool Output: 

Search results: Title: Best Places to Visit in Europe for 2025 - U.S. News Travel
Link: https://travel.usnews.com/rankings/best-europe-vacations/
Snippet: Best Places to Visit in Europe for 2025 · Swiss Alps · Rome · Venice · London, England · Paris · Florence · Amsterdam · Barcelona. #8 in Best ...
---
Title: The Best European Cities to Visit in 2025 - The Friendly Traveler
Link: https://thefriendlytraveler.com/top-european-destinations/
Snippet: The Best Spots for Nature or Adventure Seeking Travelers · Interlaken, Switzerland · Reykjavik, Iceland · Edinburgh, Scotland.
---
Title: 10 Best Countries To Visit In 2025, Per European Best Destinations
Link: https://www.forbes.com/sites/joan

Congrats! You just created your first agent today! Next - we will show how agents can be stitched together to create complex applications.